In [99]:
import re
import os

In [100]:
# global variables
eng_bigrams = None
fr_bigrams = None
ital_bigrams = None

eng_unigrams = None
fr_unigrams = None
ital_unigrams = None

In [101]:
# global variables
eng_bigrams = None
fr_bigrams = None
ital_bigrams = None

eng_unigrams = None
fr_unigrams = None
ital_unigrams = None

In [102]:
# start here
# TODO: fix directory stuff before submitting
cwd = os.getcwd()
print(cwd)

# open all the input training files
# TODO: CHANGE THIS!!!!!!
input_eng = open("/Users/emilyroller/Downloads/assignment-2-emroller-main/src/Data/Input/LangId.train.English")
input_fr = open("/Users/emilyroller/Downloads/assignment-2-emroller-main/src/Data/Input/LangId.train.French")
input_ital = open("/Users/emilyroller/Downloads/assignment-2-emroller-main/src/Data/Input/LangId.train.Italian")

# create english, french, and italian letter bigram counts

eng_bigrams = file_to_bigrams(input_eng)
fr_bigrams = file_to_bigrams(input_fr)
ital_bigrams = file_to_bigrams(input_ital)

# create english, french, and italian letter unigram counts
eng_unigrams = create_unigrams(input_eng)
fr_unigrams = create_unigrams(input_fr)
ital_unigrams = create_unigrams(input_ital)


# open input testing file
input_test = open("/Users/emilyroller/Downloads/assignment-2-emroller-main/src/Data/Validation/LangId.test")
# open output testing file (write to this)
output_test = open("/Users/emilyroller/Downloads/assignment-2-emroller-main/src/Data/Output/wordLangId.out", "w+")

line_count = 1

# iterate through input file and write the most likely language for each line in output file
for line in input_test.readlines():
    # choose_language returns a string: "English", "French", or "Italian"
    lang = choose_language(line)
    # write it to output file along with line number
    l = str(line_count) + lang + "\n"
    output_test.write(l)
    line_count = line_count + 1

input_eng.close()
input_fr.close()
input_ital.close()
input_test.close()
output_test.close()

ione pact , pari a 2,3 milioni di euro , è stata utilizzata a favore del trasporto marittimo a corto raggio combinato .', 'nel 2000 tale quota aumenterà ; è nostra intenzione , infatti , portarla al 50 percento .', "la direzione generale competente intende mantenere questa linea d ' azione , in modo che il trasporto marittimo a corto raggio continui a fruire della giusta quota della dotazione pact .", "in occasione dell ' imminente revisione del programma , si potrebbe anche arrivare ad incorporare tale principio nel testo ufficiale .", "l ' onorevole purvis mi ha chiesto di trasmettere il suo invito alla mia collega , commissario signora de palacio .", "lo farò senz ' altro e la signora apprezzerà certamente la prospettiva di una romantica gita in scozia , che peraltro sarebbe ancora più romantica se l ' onorevole purvis volesse cedere a me il piacere di accompagnare il commissario de palacio .", 'onorevole peijs , la ringrazio per il lavoro svolto e per avere confermato che tutti ci 

In [103]:
def create_unigrams(infile):
    # so it can be read again
    infile.seek(0)
    lines = []
    for line in infile.readlines():
            # strip all punctionation from each lowercased word
        no_punc = re.sub('.\n', '', line.lower())
        lines.append(no_punc)
    
    # create unigram dictionary
    unigrams = {}
    for l in lines:
        for i in range(0, len(l)):
            bg = (l[i])

            if bg not in unigrams.keys():
                unigrams[bg] = 1  
            else:
                unigrams[bg] = unigrams[bg] + 1

    return unigrams 

In [104]:
# calls create bigrams
# use this function when creating the bigram models from the training files 
def file_to_bigrams(infile):
    lines = []
    for line in infile.readlines():
            # strip all punctionation from each lowercased word
        no_punc = re.sub('.\n', '', line.lower())
        lines.append(no_punc)

    print(lines)
    return create_bigrams(lines)


In [105]:
# trains the model for a language
# creates a dictionary of bigrams with keys = (bigram tuple), value = count
# i.e. one entry might be ('t', 'h'): 3
def create_bigrams(lines):
    
    # create bigram ditionary
    bigrams = {}
    for w in lines:
        for i in range(0, len(w) - 1):
            bg = ((w[i], w[i+1]))

            if bg not in bigrams.keys():
                bigrams[bg] = 1  
            else:
                bigrams[bg] = bigrams[bg] + 1

    return bigrams 


In [106]:
# calculate the probabilities of each language for the input line
# and returns a string form of the language that is most likely
def choose_language(line):

    # calculate probabilities of line being the given language
    eng_prob = calc_probability(line, eng_bigrams, eng_unigrams)
    fr_prob = calc_probability(line, fr_bigrams, fr_unigrams)
    ital_prob = calc_probability(line, ital_bigrams, ital_unigrams)

    # find highest likelihood language
    highest_prob = max(eng_prob, fr_prob, ital_prob)

    # return the language with the highest likelihood
    if highest_prob == eng_prob:
        return " English"
    
    if highest_prob == fr_prob:
        return " French"

    if highest_prob == ital_prob:
        return " Italian"

    # shouldn't be here....
    return ":("

In [107]:
def calc_probability(line, bigrams, unigrams):
    words = []
    for word in line.split():
            # strip all punctionation from each lowercased word
            no_punc = re.sub('[^a-zA-Z]+', '', word.lower())
            words.append(no_punc)
    
    line_bigrams = create_bigrams(words)
    
    probabilities = {}
    for bigram in line_bigrams.keys():
        letter1 = bigram[0]
        # calculating probabilities using NO smoothing
        if bigram in bigrams and letter1 in unigrams:
            probabilities[bigram] = bigrams[bigram]  / unigrams[letter1] 
        else:
            probabilities[bigram] = 0

    sum_prob = 0
    for k in probabilities.keys():
        sum_prob = sum_prob + probabilities[k]
    
    return sum_prob